In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50V2

In [ ]:
# GPU configuration
!nvidia-smi

Fri Aug  2 04:23:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Install required packages
!pip install tensorflow==2.12.0 tensorflow-addons==0.20.0 keras==2.12.0
!pip install typeguard==2.13.3 inflect==6.0.2
!pip install visualkeras
!pip install numpy==1.23.5

  Using cached inflect-6.0.2-py3-none-any.whl.metadata (21 kB)
Using cached inflect-6.0.2-py3-none-any.whl (34 kB)
  Attempting uninstall: inflect
    Found existing installation: inflect 7.3.1
    Uninstalling inflect-7.3.1:
      Successfully uninstalled inflect-7.3.1


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define labels and image size
labels = ["Citrus canker", "Citrus greening", "Citrus mealybugs", "Die back", "Foliage damaged", "Healthy leaf", "Powdery mildew", "Shot hole", "Spiny whitefly", "Yellow dragon", "Yellow leaves"]
img_size = 32


In [ ]:
# Data loading function
def get_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1]
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [ ]:
# Load and preprocess data
data_dir = get_data("/content/drive/Shareddrives/4IR_Research_Students/Emon/Sweet_orange")
X = np.array([i[0] for i in data_dir]) / 255.0
Y = np.array([i[1] for i in data_dir])

'NoneType' object is not subscriptable


<ipython-input-6-f9261e232503>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


In [ ]:
# Split data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, stratify=Y, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.18, random_state=8)

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape} - y_val shape: {y_val.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

x_train shape: (4187, 32, 32, 3) - y_train shape: (4187,)
x_val shape: (920, 32, 32, 3) - y_val shape: (920,)
x_test shape: (568, 32, 32, 3) - y_test shape: (568,)


In [ ]:
# Model definitions
def create_cnn_model(input_shape):
    base_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=input_shape)
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    return models.Model(inputs=base_model.input, outputs=x)

In [ ]:
def create_vit_model(input_dim, num_classes=11):
    num_patches = 16
    projection_dim = 64
    num_transformer_layers = 4
    num_heads = 8
    mlp_units = [projection_dim * 2, projection_dim]

    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(num_patches * projection_dim)(inputs)
    x = layers.Reshape((num_patches, projection_dim))(x)

    for _ in range(num_transformer_layers):
        attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x, x)
        x = layers.Add()([x, attn_output])
        x = layers.LayerNormalization(epsilon=1e-6)(x)

        ffn_output = layers.Dense(mlp_units[0], activation="gelu")(x)
        ffn_output = layers.Dense(mlp_units[1])(ffn_output)
        x = layers.Add()([x, ffn_output])
        x = layers.LayerNormalization(epsilon=1e-6)(x)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return models.Model(inputs, outputs)

In [ ]:
def create_integrated_model(input_shape, num_classes=11):
    cnn_model = create_cnn_model(input_shape)
    cnn_output = cnn_model.output_shape[1]
    vit_model = create_vit_model(cnn_output, num_classes)

    inputs = layers.Input(shape=input_shape)
    x = cnn_model(inputs)
    outputs = vit_model(x)

    return models.Model(inputs, outputs)

In [ ]:
# Create and compile the model
input_shape = (32, 32, 3)
num_classes = 11
integrated_model = create_integrated_model(input_shape, num_classes)
integrated_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

94668760/94668760 [==============================] - 1s 0us/step


In [ ]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


In [ ]:
# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5),
]


In [ ]:
# Train the model
history = integrated_model.fit(
    data_augmentation(x_train),
    y_train,
    validation_data=(x_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=callbacks
)


Epoch 1/100
131/131 [==============================] - 313s 2s/step - loss: 2.3198 - accuracy: 0.1634 - val_loss: 2.1604 - val_accuracy: 0.2098 - lr: 1.0000e-04
Epoch 2/100
131/131 [==============================] - 279s 2s/step - loss: 1.8307 - accuracy: 0.3427 - val_loss: 1.7025 - val_accuracy: 0.3946 - lr: 1.0000e-04
Epoch 3/100
131/131 [==============================] - 283s 2s/step - loss: 1.4829 - accuracy: 0.4576 - val_loss: 1.3948 - val_accuracy: 0.4848 - lr: 1.0000e-04
Epoch 4/100
131/131 [==============================] - 278s 2s/step - loss: 1.2190 - accuracy: 0.5594 - val_loss: 1.0807 - val_accuracy: 0.6174 - lr: 1.0000e-04
Epoch 5/100
131/131 [==============================] - 284s 2s/step - loss: 1.0411 - accuracy: 0.6324 - val_loss: 1.0245 - val_accuracy: 0.6250 - lr: 1.0000e-04
Epoch 6/100
131/131 [==============================] - 278s 2s/step - loss: 0.8529 - accuracy: 0.6984 - val_loss: 0.9015 - val_accuracy: 0.6946 - lr: 1.0000e-04
Epoch 7/100
131/131 [=============

In [ ]:
# Evaluate the model
train_loss, train_accuracy = integrated_model.evaluate(x_train, y_train)
test_loss, test_accuracy = integrated_model.evaluate(x_test, y_test)

print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, labels, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')
    plt.title(title)
    plt.tight_layout()
    plt.show()

In [ ]:
# Validation set predictions and metrics
y_val_pred = integrated_model.predict(x_val)
y_val_pred_classes = np.argmax(y_val_pred, axis=1)

print("Validation Set Classification Report:")
print(classification_report(y_val, y_val_pred_classes, target_names=labels))

plot_confusion_matrix(y_val, y_val_pred_classes, labels, 'Validation Set Confusion Matrix')


In [ ]:
# Test set predictions and metrics
y_test_pred = integrated_model.predict(x_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)

print("Test Set Classification Report:")
print(classification_report(y_test, y_test_pred_classes, target_names=labels))

plot_confusion_matrix(y_test, y_test_pred_classes, labels, 'Test Set Confusion Matrix')